## Importar bibliotecas y cadena

In [1]:
import re
import time
from PIL import Image, ImageDraw
import numpy as np

In [3]:
cadena0 = open('sequence.txt','r')
codificante = list(cadena0.read())
print(len(codificante))

102655167


### Encontrar donde empiezan los nucleotidos y unir las cadenas

In [4]:
codificante[1082:1084]

['N', 'A']

In [5]:
Genoma = re.findall('[ACGT]+', "".join(codificante[1083:]))
Genoma = "".join(Genoma)

# Crear las Subsecuencias

In [6]:
def crear_subsecuencias(cadena, longitud):
    subsecuencias = []
    for i in range(0, len(cadena)-longitud+1):
        subsecuencias.append(cadena[i:i+longitud])
    return subsecuencias

In [7]:
lista_subseq = crear_subsecuencias(Genoma, 7)

## Organizar alfabeticamente y encontrar frecuencias

In [8]:
unicos = set(lista_subseq)
unicos = sorted(unicos)
lista_subseq = sorted(lista_subseq)
dic_frecuencias_2 = {}

In [9]:
def frecuencia_optimizada(lista_subsecuencias, set_unicos):
    i = 1
    contador = 1
    for subseq in set_unicos:
        if subseq == set_unicos[-1]:
            dic_frecuencias_2[subseq] = lista_subsecuencias[i-1:].count(subseq)
        else:
            while lista_subsecuencias[i] == lista_subsecuencias[i-1]:
                contador += 1
                i += 1
            else:
                dic_frecuencias_2[subseq] = contador
                i += 1
                contador = 1

In [10]:
start = time.time()
frecuencia_optimizada(lista_subseq, unicos)
end = time.time()
print(end-start)

101.15144681930542


## Crear y escribir el vector de frecuencias en un archivo

In [11]:
vector = list(dic_frecuencias_2.values())
#with open('Vectores/prueba.txt', 'w') as filehandle:
#    for num in vector:
#        filehandle.write('%s\n' % num)

# Crear CGR

In [2]:

def CGR(n, frecuencias):
    width = 1280
    height = 1280
    im = Image.new('HSV', (width,height), (0,0,0))
    draw = ImageDraw.Draw(im)
    x_divisions = [int(width*i/2**n) for i in range(2**n+1)]
    y_divisions = [int(height*i/2**n) for i in range(2**n+1)]
    maximo = max(frecuencias)
    contador=0
    for i in range(2**n):
        for j in range(2**n):
            hue = 120
            m = np.mean(frecuencias)
            if frecuencias[contador]<= m:
                saturation = int(frecuencias[contador]*255/m)
                value = 255
                draw.rectangle([(x_divisions[j], y_divisions[i]),(x_divisions[j+1], y_divisions[i+1])],(hue, saturation, value))
                contador+=1
            else:
                saturation = 255
                value = 255 - int((frecuencias[contador]-int(m))*255/(maximo-int(m)))
                draw.rectangle([(x_divisions[j], y_divisions[i]),(x_divisions[j+1], y_divisions[i+1])],(hue, saturation, value))
                contador+=1
                
    im.show()

print("done")

def incrementos(n):
    if n==1 or n==0:
        return 2
    else: return 4**(n-1) + incrementos(n-1) 

    
def acomodar(n, frecuencias, index=0):
    """
    

    Parameters
    ----------
    n : TYPE int
        DESCRIPTION. longitud de las cadenas de ADN
    frecuencias : TYPE list
        DESCRIPTION. vector de frecuencias en orden alfabético
    index: TYPE int
        Description. indice en donde comienza el renglon

    Returns
    -------
    frec : TYPE list
        DESCRIPTION. lista con elementos del primer renglon del cuadro CGR

    """
    frec = []
    contador = 1
    frec.append(frecuencias[index])
    for i in range(1, len(frecuencias)):
        for k in range(2, n+1):
            if contador in list(range(2**(k-1),1000,2**k)):
                index += incrementos(k)
                contador += 1
                try:
                    frec.append(frecuencias[index])
                except:
                    pass
                break
            elif contador%2 != 0: 
                index += 2
                try:
                    frec.append(frecuencias[index])
                except:
                    pass
                contador += 1
                break
    
    return frec
print("done")
def completar(n, frecuencias):
    frec=[]
    contador=1
    index=0
    for i in range(2**n):
        for k in range(2, n+2):
            if contador in list(range(2**(k-1),1000,2**k)):
                frec = frec + acomodar(n, frecuencias, index=index)
                index += int(incrementos(k)/2)
                contador += 1
                break
            elif contador%2 != 0:
                frec = frec + acomodar(n, frecuencias, index=index)
                index += 1
                contador += 1
                break
                
    return frec
print("done")
def crear_cgr(n, vector_frecuencias):
    frecuencias_acomodadas = completar(n, vector_frecuencias)
    CGR(n, frecuencias_acomodadas)

done
done
done


In [ ]:
crear_cgr(7, vector)